# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770939911031                   -0.53    9.0         
  2   -2.772146955010       -2.92       -1.32    1.0    233ms
  3   -2.772170503651       -4.63       -2.53    1.0    182ms
  4   -2.772170681081       -6.75       -3.32    1.0    180ms
  5   -2.772170720258       -7.41       -3.81    2.0    250ms
  6   -2.772170722968       -8.57       -5.42    1.0    189ms
  7   -2.772170723014      -10.34       -5.43    3.0    263ms
  8   -2.772170723015      -11.81       -6.53    1.0    255ms
  9   -2.772170723015      -13.52       -6.96    2.0    243ms
 10   -2.772170723015   +  -13.83       -7.51    1.0    215ms
 11   -2.772170723015      -13.73       -8.15    1.0    220ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770862908379                   -0.53    9.0         
  2   -2.7

1.7735578997794241

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770669531477                   -0.53    8.0         
  2   -2.772053655492       -2.86       -1.31    1.0    175ms
  3   -2.772082894113       -4.53       -2.61    1.0    227ms
  4   -2.772083413377       -6.28       -3.86    2.0    209ms
  5   -2.772083417328       -8.40       -4.22    2.0    217ms
  6   -2.772083417797       -9.33       -5.26    1.0    466ms
  7   -2.772083417810      -10.88       -6.06    2.0    293ms
  8   -2.772083417811      -12.93       -6.47    2.0    332ms
  9   -2.772083417811   +  -13.97       -7.21    1.0    810ms
 10   -2.772083417811      -13.75       -8.37    2.0    256ms

Polarizability via ForwardDiff:       1.7725349639324037
Polarizability via finite difference: 1.7735578997794241
